# LangChain 快速入門
[LangChain](https://github.com/langchain-ai/langchain) 是針對大型語言模型 (LLM) 應用的相關實作加以抽象化的程式語言框架，支援 Python 與 Node.js 兩種程式語言，也能夠支援包含 OpenAI GPT 等多種 LLM ，在 LangChain 中不僅僅限於 LLM 之呼叫，還能納入一系列相關工具的呼叫。藉由 LangChain 所提供的 Chain 之標準介面，目前已經有著大量工具與資料來源能夠整合至 LangChain 框架之內。鍊是一種通用的概念，可以將特定運算簡化為一個可以重複使用；包含輸入/處理/輸出的一個元件，下圖就是 LangChain 最常用的鏈 LLMChain 為例，它結合了 PromptTemplate 獲取用戶輸入，支援多種大型語言模型，並將大型語言模型輸出的內容透過 Output Parser 轉換為特定格式， Chain 也可以將輸出再傳遞給其他鍊組合出更複雜的應用。

![Chain 概念，以 LLMChain 為例](./assets/chain-concept.png)

載入會使用到的套件

In [7]:
%pip install langchain openai python-dotenv wikipedia faiss-cpu

Note: you may need to restart the kernel to use updated packages.


## 1. 運用輔助函式庫，從環境變數取得 Azure OpenAI API 相關資訊

開始使用 LangChain，Azure OpenAI Service 的主要參數都是透過環境變數設定完成，只需要指定模型之 Deployment_Name。其餘連線必要資訊皆透過以下四個環境變數指定即可

+ OPENAI_API_TYPE
+ OPENAI_API_VERSION
+ OPENAI_API_BASE
+ OPENAI_API_KEY

In [1]:
import os
from dotenv import load_dotenv
from langchain.llms import AzureOpenAI

# 載入環境變數
load_dotenv()

# 設定呼叫 OpenAI API 所需連線資訊
model = os.getenv("DEPLOYMENT_NAME")

## 2. 最簡單的自動完成範例


In [46]:
# 目前 LangChain 的 Azure OpenAI LLM 是封裝 Azure OpenAI Completeion API 而成的，搭配 GPT-3.5 仍會有自動完成無法結束的問題

llm = AzureOpenAI(deployment_name=model,model_name='gpt-35-turbo')
llm ('Q:世界人口最多的兩個國家?\nA:',stop ="\n")

' 中國、印度'

In [47]:
print (llm)

AzureOpenAI
Params: {'deployment_name': 'gpt-35-turbo', 'model_name': 'gpt-35-turbo', 'temperature': 0.7, 'max_tokens': 256, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'request_timeout': None, 'logit_bias': {}}


使用 [AzureChatOpenAI](https://python.langchain.com/docs/integrations/chat/azure_chat_openai) 則會使用 ChatComplete API 來完成對話。

In [48]:
from langchain.chat_models import AzureChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
chat = AzureChatOpenAI(deployment_name=model,model_name='gpt-35-turbo')
messages = [
    SystemMessage(
        content="你是一個回覆正體中文的機器人"
    ),
    HumanMessage(
        content="世界人口最多的兩個國家?"
    ),
]
responses = chat(messages)
print(responses.content)

世界人口最多的兩個國家為中國和印度。


接下嘗試使用 Chain 來處理相同問題，LLMChain 是最基本的 Chain，能夠以 PromptTemplate 增加輸入提示時的彈性，並將中間處理的大型語言模型抽象化，最後視狀況可以搭配多種 OutputParser 來處理輸出。由接下來的範例可以看出，我們可將 LLM Model 的 AzureOpenAI 與 Chat Model 的 AzureChatOpenAI 都使用一致的 LLMChain 鍊來進行處理。

In [49]:
from langchain.llms import OpenAI
from langchain import LLMChain,PromptTemplate

llm = AzureOpenAI(deployment_name=model,model_name='gpt-35-turbo',temperature=0,model_kwargs={"stop": ["\n"]})

# 設定 PromptTemplate
prompt = PromptTemplate(input_variables=["prompt_str"],template="{prompt_str}")

# 將 AzureOpenAI 以 LLMChain 方式使用
chain = LLMChain(llm=llm,prompt=prompt)
response = chain.run("Q:世界人口最多的兩個國家?\nA:")

# 單純輸出內容是字串
print(response)
print(type(response))

 中國、印度
<class 'str'>


In [50]:
from langchain.output_parsers import CommaSeparatedListOutputParser 

# 設定 PromptTemplate
prompt = PromptTemplate(input_variables=["prompt_str"],template="{prompt_str}")

# 設定輸出格式解析器
output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()

prompt = PromptTemplate(
    template="{prompt_str}\n{format_instructions}\nA:",
    input_variables=["prompt_str"],
    partial_variables={"format_instructions": format_instructions}
)

# 增加輸出格式解析 
chain = LLMChain(llm=llm,prompt=prompt,output_parser=output_parser)
response = chain.run("Q:世界人口最多的兩個國家?")

# 則輸出內容是以逗點隔開的 List 
print(response)
print(type(response))

['中國', '印度']
<class 'list'>


In [51]:
from langchain.chat_models import AzureChatOpenAI

chat = AzureChatOpenAI(deployment_name=model,model_name='gpt-35-turbo')

# 將 AzureChatOpenAI 以 LLMChain 方式使用
chain = LLMChain(llm=chat,prompt=prompt)
responses = chain.run("世界人口最多的兩個國家?")

# 沿用前面建立的 Prompt Template 與 Ouptut Parser 輸出內容是逗點隔開的 List 
print(response)
print(type(response))

['中國', '印度']
<class 'list'>


## 3. 使用文件載入工具載入 Wikipedia 內容

LangChain 提供豐富的文件載入 (Documenet Loader) 工具，以便將一般文字檔， CSV 格式檔案，PDF 格式檔案，JSON 格式檔案等眾多檔案格式以一致的 Document 物件來被框架內其他物件所使用，目前 LangChain 有超過七十種以上的文件載入工具可以供選用。接下來以能夠搜尋與載入 Wikipedia 內容 的 WikipediaLoader 作為範例。

In [52]:
from langchain.document_loaders import WikipediaLoader

# 查詢 Wikipedia 台灣條目，透過 load_max_docs 參數限制最多只載入兩篇內容
docs = WikipediaLoader(query="台灣", load_max_docs=2).load()
len(docs)

2

In [53]:
# 顯示第一篇內容的 Metadata
docs[0].metadata 


{'title': 'Taiwan',
 'summary': 'Taiwan, officially the Republic of China (ROC), is a country in East Asia. It is located at the junction of the East and South China Seas in the northwestern Pacific Ocean, with the People\'s Republic of China (PRC) to the northwest, Japan to the northeast, and the Philippines to the south. The territories controlled by the ROC consist of 168 islands with a combined area of 36,193 square kilometres (13,974 square miles). The main island of Taiwan, also known as Formosa, has an area of 35,808 square kilometres (13,826 square miles), with mountain ranges dominating the eastern two-thirds and plains in the western third, where its highly urbanized population is concentrated. The capital, Taipei, forms along with New Taipei City and Keelung, the largest metropolitan area in Taiwan. Other major cities include Taoyuan, Taichung, Tainan, and Kaohsiung. With around 23.9 million inhabitants, Taiwan is among the most densely populated countries in the world.\nTai

In [54]:
# 顯示第一篇內容前 400 字
content = docs[0].page_content[:400]  
print(content)

Taiwan, officially the Republic of China (ROC), is a country in East Asia. It is located at the junction of the East and South China Seas in the northwestern Pacific Ocean, with the People's Republic of China (PRC) to the northwest, Japan to the northeast, and the Philippines to the south. The territories controlled by the ROC consist of 168 islands with a combined area of 36,193 square kilometres


## 4 使用載入的文件詢問問題

以基礎模型來回答問題

In [55]:
# 完全依賴基礎模型回答問題，得到的答案是 3.6 萬平方公里
llm = AzureOpenAI(deployment_name=model,model_name='gpt-35-turbo',temperature=0,model_kwargs={"stop": ["\n"]})
prompt = "問題:台灣面積有多大?\n答案:"
response = llm (prompt ,max_tokens=1000)
print (response)

台灣面積約有3.6萬平方公里。


使用之前已經建立的 llm 來詢問 Wikipedia 下載的文件內容，由回覆的答案可以知道並非來自基礎模型，而是來自下載的 Wikipedia 文件

In [56]:
# 使用 Wikipeida 載入之文件內容回答問題，得到 36,193 平方公里
prompt = "問題:依據以下事實回答，台灣面積有多大?\n事實:"+content+"\n答案:"
print (prompt)
prediction = llm (prompt ,max_tokens=1000)
print (prediction)

問題:依據以下事實回答，台灣面積有多大?
事實:Taiwan, officially the Republic of China (ROC), is a country in East Asia. It is located at the junction of the East and South China Seas in the northwestern Pacific Ocean, with the People's Republic of China (PRC) to the northwest, Japan to the northeast, and the Philippines to the south. The territories controlled by the ROC consist of 168 islands with a combined area of 36,193 square kilometres
答案:
36,193平方公里


也可以利用 prompt template 來增加提示的彈性，prompt template 除了可以搭配 LLMChain 這個鍊使用之外，也可單純用搭配特定的大型語言模型來使用，例如 AzureOpenAI，只是需要將 PromptTemplate 物件輸出轉換為字串

In [57]:
from langchain import PromptTemplate

prompt_template = PromptTemplate(input_variables=["doc_content"], template="問題:依據以下事實回答，台灣面積有多大?\n事實:{doc_content}\n答案:")
prompt_formatted_str: str = prompt_template.format(doc_content = content)

response = llm.predict(prompt_formatted_str,max_tokens=1000,temperature=0)
print (response)

36,193平方公里


## 5. 改由使用 Chain 的方式來處理文件詢問問題
接下來我們開始運用各式 Chain 來簡化應用程式撰寫複雜度，下面程式碼是使用專為處理問答的 QA Chain 搭配 AzureOpenAI 模型來處理下載的 Wikipedia 條目文件 docs 之內容問答

In [59]:
from langchain.chains.question_answering import load_qa_chain

llm = AzureOpenAI(deployment_name=model,model_name='gpt-35-turbo',temperature=0,model_kwargs={"stop": ["\n"]})

# 建立問答鏈與載入 Wikipedia 資料
chain = load_qa_chain(llm, chain_type="stuff")
query = "台灣面積有多少平方公里?\n整數答案:"
response = chain.run(input_documents=docs, question=query, return_only_outputs=True)

# 顯示輸出結果與資料型態
print(response)
print(type(response))

 36,193
<class 'str'>


LangChain 擁有眾多功能豐富之 Chain，例如我們可以使用 [LLMMathChain](https://python.langchain.com/docs/use_cases/code_writing/llm_math) 為現有大型語言模型增添數學運算的能力，利用大型語言模型解析自然語言，再搭配 Python 來進行數學運算。 由於台灣林地面積為總面積的 58%，將前一個問答鍊的台灣總面積的答案輸入到一下個數學計算鍊，即可計算出台灣林地面積為 20991 平方公里。

In [44]:
from langchain import LLMMathChain

prompt = response +"乘上百分之五十八等於多少?" 
llm_math = LLMMathChain.from_llm(llm=llm, verbose=False)
llm_math.run( prompt)

'Answer: 20991.94'

## 6 練習在 LangChain 內使用 Azure OpenAI Embeddings
利用台北市政府開放資料 [112年度美食在台北「鍋際大賞」店家名單](https://data.taipei/dataset/detail?id=7eec2e8f-02f9-4100-9171-66f6667b8b3d) 為範例，以 LangChain 所提供的 CSV 載入工具，將資料載入後，再使用 Azure OpenAI Embeddings 進行店家名稱與地址的向量計算。並以 Meta 公司 Facebook AI 團隊開放原始程式碼向量資料庫 FAISS 將向量值儲存在本機的檔案系統中，以便後續的向量近似搜尋，FAISS 的近似比對是採用 L2 norm 方式來將兩個向量間的距離正規化，分數越接近零代表兩個向量越相似，這與 OpenAI Embeddings 的餘弦近似 (cosine similarity) 正規化後的分數值不一樣，餘弦近似分數越接近 1 的則兩個向量越相似。

In [2]:
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

# 建立 Azure OpenAI Embeddings 類別的實例
embeddings_model = OpenAIEmbeddings(deployment="text-embedding-ada-002", chunk_size = 16)

# 以 LangChain 的 CSVLoader 類別，讀取 CSV 檔案並載入文件
loader = CSVLoader(file_path="./data/2023-taipei-hot pot-restaurant.csv",
                   csv_args={
                    "delimiter": ",",                    
                    "fieldnames": ["序號","區域","店家","報名組別","營業地址","Longitude","Latitude"],
                    },                    
                   encoding="utf-8")
docs = loader.load()

# 顯示 docs 內的第一筆資料
print (docs[1].page_content)

# 載入 CSV 文件計算 OpenAI Embeddings 向量值，並建立 FAISS 向量資料庫索引
db = FAISS.from_documents(documents=docs, embedding=embeddings_model)

# 將 FAISS 向量資料索引儲存於本機磁碟 
db.save_local("./data/FAISS/faiss_index")

序號: 1
區域: 中山區
店家: 滿堂紅頂級麻辣鴛鴦鍋
報名組別: 經典鍋物組(799以下)
營業地址: 臺北市中山區松江路185號2樓
Longitude: 121.5331715
Latitude: 25.05724095


以下重新載入 FAISS 向量值索引，並開始利用 FAISS 進行向量近似比對

In [37]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

# 建立 Azure OpenAI Embeddings 類別的實例
embeddings_model = OpenAIEmbeddings(deployment="text-embedding-ada-002", chunk_size = 16)

# 如果資料沒有變更未來僅需 db = FAISS.load_local("faiss_index", embeddings_model) 方式即可載入之前建立的向量索引，無須重新計算文件的向量值
db = FAISS.load_local("./data/FAISS/faiss_index", embeddings_model)

In [38]:
# 直接傳入查詢字串方式來進行相似度搜尋
query = "士林"
docs_and_scores = db.similarity_search_with_score(query)

# 顯示 docs_and_scores 全部內容
for doc in docs_and_scores:
    print(doc)  


(Document(page_content='序號: 60\n區域: 士林區\n店家: 錢都日式涮涮鍋-士林芝山店\n報名組別: 經典鍋物組(799以下)\n營業地址: 臺北市士林區福國路71號-1\nLongitude: 121.5232239\nLatitude: 25.10250454', metadata={'source': './data/2023-taipei-hot pot-restaurant.csv', 'row': 60}), 0.40526083)
(Document(page_content='序號: 61\n區域: 士林區\n店家: 藏王極上鍋物\n報名組別: 饗宴鍋物組(800以上)\n營業地址: 臺北市士林區忠誠路二段55號3樓\nLongitude: 121.5310986\nLatitude: 25.11282254', metadata={'source': './data/2023-taipei-hot pot-restaurant.csv', 'row': 61}), 0.41024953)
(Document(page_content='序號: 59\n區域: 士林區\n店家: 川邸鍋物專門\n報名組別: 經典鍋物組(799以下)\n營業地址: 臺北市士林區承德路四段79號1樓\nLongitude: 121.5232675\nLatitude: 25.08314465', metadata={'source': './data/2023-taipei-hot pot-restaurant.csv', 'row': 59}), 0.41173244)
(Document(page_content='序號: 52\n區域: 信義區\n店家: 王鍋屋Shabu ong 京都風味酸白菜鍋專門店\n報名組別: 饗宴鍋物組(800以上)\n營業地址: 臺北市信義區逸仙路50巷20號1樓\nLongitude: 121.5626957\nLatitude: 25.04119888', metadata={'source': './data/2023-taipei-hot pot-restaurant.csv', 'row': 52}), 0.4437362)


In [39]:
# 以計算查詢字串向量方式來進行相似度搜尋
query = "滿堂紅"
embedding_vector = embeddings_model.embed_query(query)
docs_and_scores = db.similarity_search_by_vector(embedding_vector)

# 顯示 docs_and_scores 全部內容
for doc in docs_and_scores:
    print(doc)  

page_content='序號: 1\n區域: 中山區\n店家: 滿堂紅頂級麻辣鴛鴦鍋\n報名組別: 經典鍋物組(799以下)\n營業地址: 臺北市中山區松江路185號2樓\nLongitude: 121.5331715\nLatitude: 25.05724095' metadata={'source': './data/2023-taipei-hot pot-restaurant.csv', 'row': 1}
page_content='序號: 30\n區域: 松山區\n店家: 芳朵麻辣鍋 Fondue M spicy pot\n報名組別: 饗宴鍋物組(800以上)\n營業地址: 臺北市松山區光復北路98號2樓\nLongitude: 121.5574816\nLatitude: 25.05121754' metadata={'source': './data/2023-taipei-hot pot-restaurant.csv', 'row': 30}
page_content='序號: 44\n區域: 大安區\n店家: 養心殿精緻鍋物\n報名組別: 饗宴鍋物組(800以上)\n營業地址: 臺北市大安區市民大道四段110號1樓\nLongitude: 121.5505838\nLatitude: 25.04554317' metadata={'source': './data/2023-taipei-hot pot-restaurant.csv', 'row': 44}
page_content='序號: 39\n區域: 大安區\n店家: 太和殿\n報名組別: 饗宴鍋物組(800以上)\n營業地址: 臺北市大安區信義路四段315號1樓\nLongitude: 121.5560896\nLatitude: 25.03404847' metadata={'source': './data/2023-taipei-hot pot-restaurant.csv', 'row': 39}
